Main Break Sensitivity Analysis

Once the water main asset prediction/assessment function has been performed this script can be applied to review input sensitivity 
It is basically repeating a method to print results in the notebook using pandas data frame table summaries
ie query and display results using this technology

Each field of interest can be reviewed with a total count of breaks, total feet of pipe for the field value, then another table showing the value and the number of breaks per 1000ft of pipe having that value in order to gain understanding of the sensitivity of that value group to likelhood of break.

in the results we are looking at relatively high values for breaks per 1000 feet with shape length minimums having good statistically represetative groups

any factor we can think of and add, like hydrant flow static pressure, can be tested for the correlation with likelihood of break to itelligently adjust asset models and focus on scoring factors that statistically lead to higher likelihood of break

In [9]:
watermains = r"Water_Main_2022_05_26"

#make a selection for potable 

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)




<Result 'Water_Main_2022_05_26'>

In [10]:
#what is the sensitivity of soil corrosive property to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['SoilCorrosionValue', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['SoilCorrosionValue']).sum()
print(table)
table = df.groupby(['SoilCorrosionValue']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#display(HTML(table.to_html()))

                    BreakCount  MilesOfPipe  Shape_Length
SoilCorrosionValue                                       
0                          122    80.078454  4.228142e+05
3                          138    95.287942  5.031203e+05
5                          386   257.680186  1.360551e+06
SoilCorrosionValue
0    0.288543
3    0.274288
5    0.283709
dtype: float64


overall there is weak correlation here between soil corrosivity and LOB.
medium corrosivity is more likely to break than high corrosivity soils.
if medium corrosivity is more likely to break than high corrisivity should we be scoring this smiliar weight to age or material?

In [11]:
#what is the sensitivity of operational pressure zone to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['OperationArea', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['OperationArea']).sum()
print(table)
table = df.groupby(['OperationArea']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)



               BreakCount  MilesOfPipe  Shape_Length
OperationArea                                       
HP                    273   183.759161  9.702484e+05
LP                    373   249.287421  1.316238e+06
OperationArea
HP    0.281371
LP    0.283383
dtype: float64


This is a weak correlation too

In [12]:
#the pressure zone doesnt seem to make a lot of difference, what about the static pressure estimated from hydrant flow tests?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['HG_Score', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['HG_Score']).sum()
print(table)
table = df.groupby(['HG_Score']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#scores entered into histogram, most pressures are between 58-85 psi
#range is about 45-120
#60-85 = 0
#>60=1
#85.1-90 = 2
#90.1-95=3
#95.1-105=4
#>105.1 = 5

#pipes with static pressure greater than 105 psi are more likely to break
#pipes really like to be between 60-90 PSI
#we could explore the ranges a little more these ranges were based on the value histograms

          BreakCount  MilesOfPipe  Shape_Length
HG_Score                                       
0.0              378   277.558285  1.465508e+06
1.0              117    84.547428  4.464104e+05
2.0               31    18.645477  9.844812e+04
3.0               19    13.227088  6.983902e+04
4.0               42    21.012247  1.109447e+05
5.0               59    17.602884  9.294323e+04
HG_Score
0.0    0.257931
1.0    0.262091
2.0    0.314887
3.0    0.272054
4.0    0.378567
5.0    0.634796
dtype: float64


In [14]:
#This is to look at how materials perform under high static pressure

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['HG_Score', 'Material', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['HG_Score', 'Material']).sum()
print(table)
table = df.groupby(['HG_Score', 'Material']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#scores entered into histogram, most pressures are between 58-85 psi
#range is about 45-120
#60-85 = 0
#>60=1
#85.1-90 = 2
#90.1-95=3
#95.1-105=4
#>105.1 = 5

#PVC does well at high pressure but it is also newer


                     BreakCount  MilesOfPipe   Shape_Length
HG_Score Material                                          
0.0      CAS                194    43.124462  227697.161962
         COP                 12     6.028510   31830.534373
         DIP                 78    49.699738  262414.617787
         GP                   3     0.214813    1134.212754
         NIL                  0     0.196735    1038.762957
         PE                   0     0.885689    4676.440402
         PVC                 79   168.340982  888840.387149
         RCP                  2     6.514750   34397.881124
         SP                   4     0.360996    1906.061471
         TTE                  6     2.191607   11571.683954
1.0      CAS                 56    20.034136  105780.239720
         COP                  0     1.239095    6542.420144
         DIP                 30    14.453455   76314.241382
         GP                   0     0.002269      11.981977
         NIL                  0     0.44

In [15]:
#the pressure zone doesn seem to make a lot of difference, what about the static pressure estimated from hydrant flow tests?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = [ 'Material', 'HG_Score','BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Material','HG_Score']).sum()
print(table)
table = df.groupby(['Material','HG_Score']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#scores entered into histogram, most pressures are between 58-85 psi
#range is about 45-120
#60-85 = 0
#>60=1
#85.1-90 = 2
#90.1-95=3
#95.1-105=4
#>105.1 = 5

#pipes with static pressure greater than 105 psi are more likely to break
#pipes really like to be between 85-90 PSI
#we could explore the ranges a little more these ranges were based on a histogram I looked at


                     BreakCount  MilesOfPipe   Shape_Length
Material   HG_Score                                        
CAS        0.0              194    43.124462  227697.161962
           1.0               56    20.034136  105780.239720
           2.0               11     2.083900   11002.992938
           3.0                5     1.475553    7790.920276
           4.0               16     2.416539   12759.327031
           5.0               29     3.437826   18151.723652
COP        0.0               12     6.028510   31830.534373
           1.0                0     1.239095    6542.420144
           2.0                6     0.859403    4537.645678
           3.0                2     0.852043    4498.784991
           4.0                4     0.774121    4087.356661
           5.0                0     0.263064    1388.976940
DIP        0.0               78    49.699738  262414.617787
           1.0               30    14.453455   76314.241382
           2.0               11     4.78

In [16]:
#what is the sensitivity of operational pressure zone and install era to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['OperationArea', 'Install_Era', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['OperationArea', 'Install_Era']).sum()
print(table)
table = df.groupby(['OperationArea', 'Install_Era']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)


#pipes from the 1950 - 1980s are more likely to break in the west hills zone
# pipes from the '30s in central service zone are likely to break
# pipes from the 1900-1909 in central service zone are likely to break
# THe LOB Trend is about linear from now to 1940

                           BreakCount  MilesOfPipe   Shape_Length
OperationArea Install_Era                                        
HP            1850-1899             0     0.002911      15.371373
              1920-1929             2     0.501112    2645.870235
              1930-1939             1     0.295203    1558.674288
              1940-1949             0     0.183842     970.684927
              1950-1959            27     6.539635   34529.274189
              1960-1969            45    14.042327   74143.483741
              1970-1979            81    18.597066   98192.508680
              1980-1989            47    26.644892  140685.030920
              1990-1999            48    40.642039  214589.968340
              2000-2009            18    44.149169  233107.613087
              2010-2019             4    30.697058  162080.465022
              2020-2029             0     1.463907    7729.427729
LP            1850-1899             2     0.373751    1973.407082
          

In [17]:
#what is the sensitivity of pipe material to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Material', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Material']).sum()
print(table)
table = df.groupby(['Material']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#Ductile iron is not that bad
#GP and SP is the worst
#

#display(HTML(table.to_html()))

            BreakCount  MilesOfPipe  Shape_Length
Material                                         
CAS                311    72.597866  3.833167e+05
COP                 24    10.040108  5.301177e+04
DIP                151    81.208850  4.287827e+05
GP                   3     0.217471  1.148247e+03
NIL                  0     0.690101  3.643731e+03
PE                   0     1.492903  7.882529e+03
PVC                139   252.703062  1.334272e+06
RCP                  5    10.516807  5.552874e+04
SP                   4     0.360996  1.906061e+03
TTE                  9     3.153033  1.664801e+04
UNK                  0     0.022780  1.202809e+02
pvc sdr-14           0     0.042604  2.249510e+02
Material
CAS           0.811339
COP           0.452730
DIP           0.352160
GP            2.612678
NIL           0.000000
PE            0.000000
PVC           0.104177
RCP           0.090043
SP            2.098568
TTE           0.540605
UNK           0.000000
pvc sdr-14    0.000000
dtype: float64


In [18]:
#what is the sensitivity of pipe material to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Material', 'PipeDiameterScore', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Material', 'PipeDiameterScore']).sum()
print(table)
table = df.groupby(['Material', 'PipeDiameterScore',]).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#Ductile iron is not that bad
#GP and SP is the worst
#

#display(HTML(table.to_html()))

                              BreakCount  MilesOfPipe   Shape_Length
Material   PipeDiameterScore                                        
CAS        0                           2     0.200931    1060.913384
           1                          18     1.657451    8751.341950
           2                         154    24.310948  128361.804252
           3                         105    28.900169  152592.891183
           4                           7     1.799921    9503.582929
           5                          22    12.917746   68205.698322
           6                           3     2.810701   14840.499803
COP        0                          22    10.003271   52817.273279
           2                           2     0.036698     193.765708
           5                           0     0.000139       0.732831
DIP        0                           0     0.004068      21.476886
           1                           0     0.051733     273.149924
           2                      

In [18]:
#what is the sensitivity of pipe material to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Material', 'PipeDiameterScore', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Material', 'PipeDiameterScore']).sum()
print(table)
table = df.groupby(['Material', 'PipeDiameterScore',]).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#Ductile iron is not that bad
#GP and SP is the worst
#

#display(HTML(table.to_html()))

                              BreakCount  MilesOfPipe   Shape_Length
Material   PipeDiameterScore                                        
CAS        0                           2     0.200931    1060.913384
           1                          18     1.657451    8751.341950
           2                         154    24.310948  128361.804252
           3                         105    28.900169  152592.891183
           4                           7     1.799921    9503.582929
           5                          22    12.917746   68205.698322
           6                           3     2.810701   14840.499803
COP        0                          22    10.003271   52817.273279
           2                           2     0.036698     193.765708
           5                           0     0.000139       0.732831
DIP        0                           0     0.004068      21.476886
           1                           0     0.051733     273.149924
           2                      

In [19]:
#what is the sensitivity of pipe diameter to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Diameter', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Diameter']).sum()
print(table)
table = df.groupby(['Diameter']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#small pipes, 1.5 in and 2 in are high LOB
#6 in pipes are high LOB
#20 in pipes are high lob

#display(HTML(table.to_html()))

          BreakCount  MilesOfPipe  Shape_Length
Diameter                                       
0.75               0     0.045281  2.390838e+02
1.00               1     0.782238  4.130216e+03
1.50               1     0.053295  2.813993e+02
2.00              34    16.640726  8.786303e+04
3.00               1     0.786160  4.150923e+03
4.00              27     6.155834  3.250280e+04
6.00             210    56.608300  2.988918e+05
8.00             269   236.186909  1.247067e+06
10.00              7     1.917197  1.012280e+04
12.00             73    83.934832  4.431759e+05
14.00              0     1.038166  5.481514e+03
16.00              8    17.880867  9.441098e+04
20.00              6     1.038535  5.483464e+03
24.00              8     8.599463  4.540516e+04
30.00              0     0.077706  4.102897e+02
36.00              1     1.301074  6.869668e+03
Diameter
0.75     0.000000
1.00     0.242118
1.50     3.553669
2.00     0.386966
3.00     0.240910
4.00     0.830698
6.00     0.702595
8

In [22]:
#what is the sensitivity of age to likelihood of main break for transmission mains?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1) AND PipeDiameterScore = 6", None)

fc = watermains

fields = ['Install_Era', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Install_Era']).sum()
print(table)
table = df.groupby(['Install_Era']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)



#display(HTML(table.to_html()))

             BreakCount  MilesOfPipe  Shape_Length
Install_Era                                       
1850-1899             0     0.009215     48.656352
1910-1919             0     1.059022   5591.635049
1950-1959             8     7.126600  37628.446235
1960-1969             0     0.597056   3152.454370
1970-1979            10     7.549887  39863.404213
1980-1989             4     3.757903  19841.726309
1990-1999             0     3.287674  17358.917376
2000-2009             0     3.130301  16527.986666
2010-2019             1     3.418154  18047.851533
Install_Era
1850-1899    0.000000
1910-1919    0.000000
1950-1959    0.212605
1960-1969    0.000000
1970-1979    0.250857
1980-1989    0.201595
1990-1999    0.000000
2000-2009    0.000000
2010-2019    0.055408
dtype: float64


In [26]:
#what is the sensitivity of pipe material to likelihood of main break for transmission mains?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1) AND PipeDiameterScore = 6", None)

fc = watermains

fields = ['Material', 'Diameter', 'BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Material', 'Diameter']).sum()
print(table)
table = df.groupby(['Material', 'Diameter',]).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#Ductile iron is not that bad
#GP and SP is the worst
#

#display(HTML(table.to_html()))

                   BreakCount  MilesOfPipe  Shape_Length
Material Diameter                                       
CAS      14.0               0     1.025815   5416.303235
         16.0               3     1.730058   9134.708677
         30.0               0     0.054827    289.487891
DIP      14.0               0     0.011904     62.851524
         16.0               3     8.078631  42655.170171
         20.0               0     0.188912    997.453273
         24.0               7     4.751764  25089.313081
         36.0               0     0.095060    501.917744
PVC      16.0               0     1.950287  10297.515707
         30.0               0     0.022879    120.801835
         36.0               1     1.206013   6367.750441
RCP      14.0               0     0.000447      2.359570
         16.0               2     6.093823  32175.384281
         20.0               2     0.574839   3035.148126
         24.0               1     3.847699  20315.850707
SP       20.0               4  

In [27]:
#the pressure zone doesn seem to make a lot of difference, what about the static pressure estimated from hydrant flow tests for Transmission Mains?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)  AND PipeDiameterScore = 6", None)

fc = watermains

fields = [ 'Material', 'HG_Score','BreakCount', 'MilesOfPipe', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Material','HG_Score']).sum()
print(table)
table = df.groupby(['Material','HG_Score']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#scores entered into histogram, most pressures are between 58-85 psi
#range is about 45-120
#60-85 = 0
#>60=1
#85.1-90 = 2
#90.1-95=3
#95.1-105=4
#>105.1 = 5

#pipes with static pressure greater than 105 psi are more likely to break
#pipes really like to be between 85-90 PSI
#we could explore the ranges a little more these ranges were based on a histogram I looked at


                   BreakCount  MilesOfPipe  Shape_Length
Material HG_Score                                       
CAS      0                  0     1.280860   6762.940818
         1                  0     0.986278   5207.546464
         2                  3     0.543563   2870.012521
DIP      0                  8     7.012156  37024.184131
         1                  2     5.091572  26883.499881
         2                  0     0.103348    545.679398
         3                  0     0.403668   2131.367349
         4                  0     0.515526   2721.975034
PVC      0                  1     3.089674  16313.477208
         1                  0     0.089506    472.590776
RCP      0                  2     6.514750  34397.881124
         1                  0     0.507257   2678.315123
         2                  0     0.151856    801.798856
         3                  0     1.680082   8870.835560
         4                  1     1.060408   5598.955298
         5                  2  